# Code for AISTATS submission (Reconstructing Test Labels from Noisy Loss Functions)
## Label inference on binary logloss using Algorithm 2 in the paper by Aggarwal et al. [5].

### Experiment setup

In [ ]:
import os
import sys
import json
import pandas as pd
import numpy as np
from collections import Counter
import math
import time
import warnings
warnings.filterwarnings('ignore')

np.random.seed(12345)

### Titanic data

In [ ]:
titanic = pd.read_csv('data/titanic.csv')
titanic = titanic.sample(frac=1)
print(titanic.shape) 
print(Counter(titanic['class']))
titanic.head()

In [ ]:
def titanic_to_int(x):
    return x if x == 1 else 0
titanic_label = titanic['class'].apply(titanic_to_int) 
Counter(titanic_label)

## utility functions in the experiments

In [ ]:
# Method to compute binary log-loss
def binary_logloss(labeling = [], pred=[], tau=0):
    """
    labeling: ground truth labels
    pred: prediction vector
    Return binary logloss score
    """
    if not pred:
        raise ValueError("No prediction vector passed")
    N = len(labeling)
    assert len(pred)==N
    loss = 0
    for i in range(N):
        theta0 = sys.float_info.min if 1 - pred[i] < sys.float_info.min else 1 - pred[i]
        loss -= labeling[i]*math.log(1-theta0) +(1-labeling[i])*math.log(theta0)
    return loss/N + tau * np.random.rand()

In [ ]:
# function to generate the predictions
def generate_prediction(N, tau_max):
    """
    N: number of data points
    tau_max: maximum noise
    Return prediction vector
    """
    prediction = np.zeros(N)
    for i in range(N):
        n = i + 1
        exponent = 2 ** n * N * tau_max
        prediction[i] = 1 / (1 + 3 ** (-exponent))
    return list(prediction)

In [ ]:
# generator for K-nary labelings
def gen_K_ary(nbits, K=2):
    """
    nbits: number of bits in the K-nary vector
    K: K in K-nary, e.g. K=2 is binary, K=3 is ternary
    Returns a generator for all K-nary labelings of length nbits
    """
    for i in range(K**nbits):
        bin_str = str(np.base_repr(i, K))
        l = len(bin_str)
        bin_arr = np.zeros(nbits)
        for j in range(l):
            bin_arr[nbits-j-1] = float(bin_str[l-j-1])
        yield [int(x) for x in bin_arr]

In [ ]:
# the main label inference algorithm
def main_algorithm(noisy_loss, N, K, prediction):
    """
    noisy_loss: observed loss score with noise
    N: number of data points
    K: number of classes
    prediction: 2-d numpy array, where prediction[i,k] represents the prediction for data i in class k
    Returns the inferred labeling
    """
    label_infer = []
    diff_min = np.inf
    lab_generator = gen_K_ary(N, K)
    
    while True:
        try:
            label_cur = next(lab_generator) 
            loss_cur = binary_logloss(label_cur, prediction, 0)
            diff_cur = np.abs(loss_cur - noisy_loss)
            # print(loss_cur, diff_cur)
            if diff_cur < diff_min:
                label_infer = label_cur.copy()
                diff_min = diff_cur
        except:
            break
    return label_infer

In [ ]:
# the main function for simulation
def main_sim(true_labeling, N, K, tau_max, nsims=1000):
    """
    true_labeling: ground truth labeling with length at least N
    N: number of data points
    K: number of classes
    tau_max: maximum noise
    nsims: number of simulations
    Returns the average accuracy and the average loss score
    """
    acc = 0
    mean_loss = 0
    for ns in range(nsims):
        labeling = list(np.random.choice(true_labeling, N))
        prediction = generate_prediction(N, tau_max)
        noisy_loss = binary_logloss(labeling, prediction, tau_max)
        mean_loss += noisy_loss
        labeling_infer = main_algorithm(noisy_loss, N, K, prediction)
        all_same = (sum([x == y for x, y in zip(labeling_infer, labeling)]) == N)
        acc += all_same
    return acc / nsims, mean_loss / nsims

## label inference experiments

In [ ]:
# logloss experiment 
nsims = 1000
result_logloss = []
K = 2

for tau_max in [1e-4, 1e-2, 1e-1, 1]:
    for N in range(1, 21):
        try:
            start = time.time()
            acc, mean_loss = main_sim(titanic_label, N, K, tau_max, nsims)
            seconds = time.time() - start
            result_logloss.append((K, tau_max, N, acc, mean_loss, seconds))
            print(K, tau_max, N, acc, mean_loss, seconds)
            if acc < 0.5: break
        except:
            break

In [ ]:
df_logloss = pd.DataFrame(result_logloss)
df_logloss.columns = ['K', 'tau_max', 'N', 'acc', 'mean_loss', 'seconds_1k']
df_logloss['ms'] = df_logloss['seconds_1k'] # since 1000 repetitions
df_logloss.to_csv('logloss_binary_icml.csv', index=False)
print(df_logloss.shape)
df_logloss.head(10)